In [1]:
def test_data(tttt):
    uttime = datetime.utcfromtimestamp(tttt).strftime('%Y-%m-%dT%H:%M:%S')
    X=api.exchange_rates_get_specific_rate('BTC', 'USD', {'time': uttime})
    return(X)               

In [ ]:
import asyncio

async def test(tttt):
    uttime = datetime.utcfromtimestamp(tttt).strftime('%Y-%m-%dT%H:%M:%S')
    X=api.exchange_rates_get_specific_rate('BTC', 'USD', {'time': uttime})
    print(X) 
# Busy waits for a second, but we don't want to stick around...
    #print('gr1 started work: {}'.format(tic()))
    #await asyncio.sleep(2)
    #print('gr1 ended work: {}'.format(tic()))
tttt=1529980664
ioloop = asyncio.get_event_loop()
tasks = [asyncio.ensure_future(test(tttt + 10*t)) for t in range(1)]
ioloop.run_until_complete(tasks)
ioloop.close()


In [9]:
mp.cpu_count()
#import multiprocessing
#This code does not improve on sending naive sequential requests.
#
#tttt= 1529969400
#jobs=[]
#print(time.time(),'start')
#for i in range(10):
#    p = multiprocessing.Process(target= test_data(tttt+10*i))
#    jobs.append(p)
#    p.start()
#print(time.time(), 'end')

8

In [8]:
#the key is mp.pool that utilizes many processes.
import multiprocessing as mp

print(time.time())
tttt= 1319090664
pool = mp.Pool(processes=7)
#for i in range(4):
print(time.time(), 'start round =')
results = [pool.apply_async(test_data, args=(tttt + t,)) for t in range(28)]
output = [p.get() for p in results]
print(output)
print(time.time(), 'end round =')
#tttt+=7

1530028523.8748128
1530028523.8998325 start round =
[{'time': '2011-10-20T06:03:22.0000000Z', 'asset_id_quote': 'USD', 'rate': 2.3259434792489566, 'asset_id_base': 'BTC'}, {'time': '2011-10-20T06:03:22.0000000Z', 'asset_id_quote': 'USD', 'rate': 2.3259434792489566, 'asset_id_base': 'BTC'}, {'time': '2011-10-20T06:03:22.0000000Z', 'asset_id_quote': 'USD', 'rate': 2.3259434792489566, 'asset_id_base': 'BTC'}, {'time': '2011-10-20T06:03:22.0000000Z', 'asset_id_quote': 'USD', 'rate': 2.3259434792489566, 'asset_id_base': 'BTC'}, {'time': '2011-10-20T06:03:22.0000000Z', 'asset_id_quote': 'USD', 'rate': 2.3259434792489566, 'asset_id_base': 'BTC'}, {'time': '2011-10-20T06:03:22.0000000Z', 'asset_id_quote': 'USD', 'rate': 2.3259434792489566, 'asset_id_base': 'BTC'}, {'time': '2011-10-20T06:03:22.0000000Z', 'asset_id_quote': 'USD', 'rate': 2.3259434792489566, 'asset_id_base': 'BTC'}, {'time': '2011-10-20T06:03:22.0000000Z', 'asset_id_quote': 'USD', 'rate': 2.3259434792489566, 'asset_id_base': 'BT

In [ ]:
#class HelloWorld(luigi.Task):
#    def requires(self):
#        return None
#    def output(self):
#        return luigi.LocalTarget('helloworld.txt')
#    def run(self):
#        with self.output().open('w') as outfile:
#            outfile.write('Hello World!\n')

#class NameSubstituter(luigi.Task):
#    name = luigi.Parameter()
#    def requires(self):
#        return HelloWorld()
#    def output(self):
#        return luigi.LocalTarget(self.input().path + '.name_' + self.name)
#    def run(self):
#        with self.input().open() as infile, self.output().open('w') as outfile:
#            text = infile.read()
#            text = text.replace('World', self.name)
#            outfile.write(text)

In [2]:
import luigi
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/fbuonerba/codes')
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request

test_key = 'DB318A59-25FF-499E-9A6D-783A19C346D8'

#Free key for testing: test_key = 'EED0F746-36FB-4CC4-8E7C-527333DFA6FB'

api = CoinAPIv1(test_key)

#this function computes how many seconds from now till midnight.
#the correction is due to the fact that some times are not utc by default.
#This can be checked transforming output into utc time.
def until_midnight():
    tomorrow = datetime.now() + timedelta(1)
    midnight = datetime(year=tomorrow.year, month=tomorrow.month, 
                        day=tomorrow.day, hour=0, minute=0, second=0)
    wrong_number=(midnight - datetime.now()).seconds
    return (wrong_number + 61)# - 14399)


In [ ]:
#this task downloads data (by luigi default, if it does not exist already).
class request_exchange_rates(luigi.Task):
    unix_time=luigi.Parameter()
    base=luigi.Parameter()
    quote=luigi.Parameter()
    path='/home/fbuonerba/exchange_rates_data/'
    def requires(self):
        return None
    def output(self):
        return luigi.LocalTarget(self.path+'exchange_rate_'+str(self.base)+'_'+str(self.quote)+'_'+str(self.unix_time)+'.txt')
    def run(self):
        while True:
            try: 
                utctime = datetime.utcfromtimestamp(self.unix_time).strftime('%Y-%m-%dT%H:%M:%S')
                exchange=api.exchange_rates_get_specific_rate(self.base, self.quote, {'time': utctime})
                with self.output().open('w') as outfile:
                    json.dump(exchange, outfile) 
                print('okokokokokok', self.unix_time)
                break
            except urllib.error.HTTPError as err:
                print('AAAAAAAAAAAAAAAAAAAA', err, self.unix_time)
                time.sleep(30)
            
        
    

In [ ]:
unix_time= 1529985000
#1586918800  1529954000
while True:
    luigi.build([request_exchange_rates(unix_time + 600*t, base='ETH', quote='USD') for t in range(2)], workers=2, local_scheduler=True)
    unix_time+=1200
    #luigi.build([request_exchange_rates(unix_time + 600*t, base='ETH', quote='USD') for t in range(100)], workers=1, local_scheduler=True)
    